# Hello World on an FPGA
Using the Lattice iCE40-HX1K iCEstick ([latticestore](http://www.latticestore.com/searchresults/tabid/463/searchid/1/searchvalue/ice40hx1k-stick-evn/default.aspx))

Download and install the icestorm toolchain at http://www.clifford.at/icestorm/ (How to Install the Tools section).

We begin by importing our Python dependencies.

In [1]:
from magma import *
from mantle import Mux, Counter, CounterModM
from loam.boards.icestick import IceStick
import silica

import magma
import lattice ice40
import lattice mantle40


We define some constants that we will use later on.

In [2]:
BAUD_RATE  = 115200
CLOCK_RATE = int(12e6)  # 12 mhz

We define a function to generate a ROM block with the parameters:
* `logn` - width of the address
* `init` - a sequence of values used to initialize the memory
* `address` - an `Out(Array(logn, Bit))` that is used as the select input to the ROM

In [3]:
def ROMNx8(init, A):
    n = len(init)
    logn = n.bit_length() - 1
    assert len(A) == logn

    muxs = [Mux(2, 8) for i in range(n - 1)]
    for i in range(n // 2):
        muxs[i](init[2*i], init[2*i+1], A[0])

    k = 0
    l = 1 << (logn-1)
    for i in range(logn-1):
        for j in range(l//2):
            muxs[k+l+j](muxs[k+2*j], muxs[k+2*j+1], A[i+1])
        k += l
        l //= 2

    return muxs[n-2]


@silica.fsm(clock_enable=True)
def uart_transmitter(data : In(Array(8, Bit)), valid : In(Bit), tx : Out(Bit)):
    while True:
        if valid:
            tx = 0  # start bit
            yield
            for i in range(0, 8):
                tx = data[i]
                yield
            tx = 1  # end bit
            yield
            # Holding the line high for an extra baud cycle seems to improve
            # reliability of transmission (reduces framing issues?)
            yield
        else:
            tx = 1
            yield


icestick = IceStick()
icestick.Clock.on()
icestick.TX.output().on()
main = icestick.main()
baud_clock = CounterModM(103, 8)
uart = uart_transmitter()

# We use a ROM to store our message, it should look up the next entry every 11
# baud ticks (the time it takes to send the current byte)
advance = CounterModM(11, 4, ce=True)
wire(advance.CE, baud_clock.COUT)

# This counter controls the ROM, it's clock enable is controlled by the baud
# clock divider `advance`
counter = Counter(4, ce=True)
wire(counter.CE, advance.COUT)

message = "Hello, world! \r\n"
message_bytes = [int2seq(ord(char), 8) for char in message]
rom = ROMNx8(message_bytes, counter.O)

wire(uart.data, rom.O)
wire(uart.valid, 1)
wire(uart.tx, main.TX)
wire(uart.CE, baud_clock.COUT)

compile("hello_world", main)

compiling Addcout8
compiling Register8R
compiling Counter8R
compiling CounterModM8
compiling Register10CE
compiling Register3CE
compiling Or2x3
compiling Or3x3
compiling Register1CE
compiling Or2x1
compiling Or3x1
compiling Cascade3x2_2
compiling And2x1
compiling Invert1
compiling Invert3
compiling Addcincout3
compiling And2x3
compiling Addcout3
compiling Mux4
compiling Mux8
compiling uart_transmitter
compiling Addcout4
compiling Register4CER
compiling Counter4CER
compiling CounterModM4CE
compiling Register4CE
compiling Counter4CE
compiling Mux2x8
compiling main


In this directory, run `make upload` (`make upload_mac` on MacOS) to program the icestick.

Run `ls /dev/ | grep usb` to find the right serial port (note you may have to try a couple).

In [4]:
import serial
import time
import sys

# argument should be something like
# /dev/tty.usbserial-142B
# /dev/ttyUSB1
with serial.Serial("/dev/tty.usbserial-1431B", BAUD_RATE, timeout=1) as ser:
    for i in range(10):
        try:
            print(ser.read(len(message) * 5).decode(), end="")
        except UnicodeDecodeError as e:
            print("Experienced {}".format(str(e)))
            print("Possible framing issue, continueing")

ello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
Hello, world! 
H